# Data Extraction

## Parsing Y- axis Ticks and Y- vals 
This code parses the x and y axes (horizontal and vertical axes) detection for the y-ticks, calculates the ratio to normalize the figure contents, specifically the bar plots with respect to the y-ticks and finally connects the evaluated y-values with their correspinding legends using clustering algorithm. 
* Note that all the figures recorded have their respective dois as a unique key of identification.

In [1]:
import cv2, imutils, re
import xlsxwriter, json, os
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from matplotlib import rcParams

In [2]:
# Directory of images to run the code on
img_dir = 'AllData/processed_images/Part2'
data_dir = 'data'

# Directory to save the output images
save_dir = 'out'

### Using json to parse the captions and dois for each image

In [3]:
with open('urldata.json') as handler:
    urldata = json.load(handler)

In [4]:
with open('doi_data.json') as doihandler:
    doidata = json.load(doihandler)

In [5]:
with open('ocr-image-text.json') as ocrhandler:
    images_text = json.load(ocrhandler)

### Function to detect x-axis and y-axis for a plot

In [6]:
def findMaxConsecutiveOnes(nums) -> int:
    count = maxCount = 0
    
    for i in range(len(nums)):
        if nums[i] == 1:
            count += 1
        else:
            maxCount = max(count, maxCount)
            count = 0
                
    return max(count, maxCount)

In [7]:
def detectAxes(filepath, threshold=None, debug=False):
    if filepath is None:
        return None, None
    
    if threshold is None:
        threshold = 10
    
    image = cv2.imread(filepath)
    height, width, channels = image.shape
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    maxConsecutiveOnes = [findMaxConsecutiveOnes(gray[:, idx] < 200) for idx in range(width)]
    start_idx, maxindex, maxcount = 0, 0, max(maxConsecutiveOnes)
    while start_idx < width:
        if abs(maxConsecutiveOnes[start_idx] - maxcount) <= threshold:
            maxindex = start_idx
            break
            
        start_idx += 1
           
    yaxis = (maxindex, 0, maxindex, height)
    
    if debug:
        fig, ax = plt.subplots(1, 2)

        ax[0].imshow(image)

        ax[1].plot(maxConsecutiveOnes, color = 'k')
        ax[1].axhline(y = max(maxConsecutiveOnes) - 10, color = 'r', linestyle = 'dashed')
        ax[1].axhline(y = max(maxConsecutiveOnes) + 10, color = 'r', linestyle = 'dashed')
        ax[1].vlines(x = maxindex, ymin = 0.0, ymax = maxConsecutiveOnes[maxindex], color = 'b', linewidth = 4)

        plt.show()

    maxConsecutiveOnes = [findMaxConsecutiveOnes(gray[idx, :] < 200) for idx in range(height)]
    start_idx, maxindex, maxcount = 0, 0, max(maxConsecutiveOnes)
    while start_idx < height:
        if abs(maxConsecutiveOnes[start_idx] - maxcount) <= threshold:
            maxindex = start_idx
            
        start_idx += 1
            
    cv2.line(image, (0, maxindex), (width, maxindex),  (255, 0, 0), 2)
    xaxis = (0, maxindex, width, maxindex)
    
    if debug:
        rcParams['figure.figsize'] = 15, 8

        fig, ax = plt.subplots(1, 1)
        ax.imshow(image, aspect = 'auto')
        
    return xaxis, yaxis

In [8]:
def getProbableLabels(image, image_text, xaxis, yaxis):
    y_labels = []
    x_labels = []
    legends = []
    y_text_list = []
    
    height, width, channels = image.shape
    
    (x1, y1, x2, y2) = xaxis
    (x11, y11, x22, y22) = yaxis
    
    for text, (textx, texty, w, h) in image_text:
        text = text.strip()
            
        # To the left of y-axis and top of x-axis
        if (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == 1):
            
            numbers = re.findall(r'\b\d+(?:\.\d+)?', text)
            if bool(numbers):
                y_labels.append((text, (textx, texty, w, h)))
            else:
                y_text_list.append((text, (textx, texty, w, h)))
            
        # To the right of y-axis and bottom of x-axis
        elif (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == 1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == -1):
            x_labels.append((text, (textx, texty, w, h)))
            
        # Top of x-axis and to the right of y-axis
        elif (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == -1):
            
            # Consider non-numeric only for legends
            if not bool(re.findall(r'\b[\d\.\d]+\b', text)):
                legends.append((text, (textx, texty, w, h)))
    
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, (text, rect) in enumerate(y_labels):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(y_labels[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    y_labels_list = maxList.copy()
    
    y_labels = []
    for text, (textx, texty, w, h) in maxList:
        y_labels.append(text)
        
    # Get the x-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(y1, height):
        count = 0
        current = []
        for index, (text, rect) in enumerate(x_labels):
            if lineIntersectsRectY(i, rect):
                count += 1
                current.append(x_labels[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    x_labels = []
    for text, (textx, texty, w, h) in maxList:
        x_labels.append(text)
    
    # Get possible legend text
    # For this, we need to search both top to
    # bottom and also from left to right.
    
    #print(legends)
    legends = mergeTextBoxes(legends)
    #print("=====================")
    #print(legends)
    #print("**********************")
    #print("**********************")
    
    maxIntersection = 0
    maxList = []
    for i in range(y1):
        count = 0
        current = []
        for index, (text, rect) in enumerate(legends):
            if lineIntersectsRectY(i, rect):
                count += 1
                current.append(legends[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
            
    for i in range(x11, width):
        count = 0
        current = []
        for index, (text, rect) in enumerate(legends):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(legends[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
        
    #print(maxList)
    #maxList = mergeTextBoxes(maxList)
    
    legends = []
    #legendBoxes = []
    for text, (textx, texty, w, h) in maxList:
        legends.append(text)
    #    legendBoxes.append((textx, texty, w, h))
        #cv2.rectangle(image, (textx, texty), (textx + w, texty + h), (255, 0, 255), 2)
    
    #legendBoxes = mergeRects(legendBoxes)
    
    #for (textx, texty, w, h) in legendBoxes:
    #    cv2.rectangle(image, (textx, texty), (textx + w, texty + h), (255, 0, 255), 2)
    
    return image, x_labels, y_labels, y_labels_list, y_text_list, legends, maxList

In [9]:
def lineIntersectsRectX(candx, rect):
    (x, y, w, h) = rect
    
    if x <= candx <= x + w:
        return True
    else:
        return False
    
def lineIntersectsRectY(candy, rect):
    (x, y, w, h) = rect
    
    if y <= candy <= y + h:
        return True
    else:
        return False

## Getting the Ratio for y-value matching 

Similar to the label detection logic, y-ticks are detected: 

Y-ticks 

*  Check only the numerical boxes which are to the left of y-axis and to the top of x-axis.
*  Run a line sweep from left end of the image to the y-axis position, and check when the sweeping line intersects with the maximum number of numerical boxes.
* The numerical boxes are then used as bounding boxes for calculating the y-ticks.

Ratio:
* Difference between the y-ticks is then calculated.  
* Only consider the mean difference between the y-ticks, rejecting the outliers from the calculated values.
* The final ratio is: <br> ``normalize_ratio := ticks_diff.mean() / y-ticks.mean()``.

In [10]:
def getRatio(path, xaxis, yaxis):
    list_text = []
    list_ticks = []
    
    filepath = img_dir + "/" + path.name
    
    image = cv2.imread(filepath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width, channels = image.shape

    image_text = images_text[path.name]
    
    for text, (textx, texty, w, h) in image_text:
        text = text.strip()
                    
        (x1, y1, x2, y2) = xaxis
        (x11, y11, x22, y22) = yaxis
        
        # To the left of y-axis and top of x-axis
        if (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == 1):
            
            # Consider numeric only for ticks on y-axis
            numbers = re.findall(r'\d+(?:\.\d+)?', text)
            if bool(numbers):
                list_text.append((numbers[0], (textx, texty, w, h)))
                          
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, (text, rect) in enumerate(list_text):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(list_text[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    # Get list of text and ticks
    list_text = []
    for text, (textx, texty, w, h) in maxList:
        list_text.append(float(text))
        list_ticks.append(float(texty + h))
        
    text_sorted = (sorted(list_text))
    ticks_sorted  = (sorted(list_ticks))
    
    ticks_diff = ([ticks_sorted[i] - ticks_sorted[i-1] for i in range(1, len(ticks_sorted))])
    text_diff = ([text_sorted[i] - text_sorted[i-1] for i in range(1, len(text_sorted))])
    print("[get text-to-tick ratio] ticks_diff: {0}, text_diff: {1}".format(ticks_diff, text_diff))
    
    # Detected text may not be perfect! Remove the outliers.
    ticks_diff = reject_outliers(np.array(ticks_diff), m=1)
    text_diff = reject_outliers(np.array(text_diff), m=1)
    print("[reject_outliers] ticks_diff: {0}, text_diff: {1}".format(ticks_diff, text_diff))
    
    normalize_ratio = np.array(text_diff).mean() / np.array(ticks_diff).mean()

    return text_sorted, normalize_ratio

In [11]:
def lineIntersectsRectX(candx, rect):
    (x, y, w, h) = rect
    
    if x <= candx <= x + w:
        return True
    else:
        return False
    
def lineIntersectsRectY(candy, rect):
    (x, y, w, h) = rect
    
    if y <= candy <= y + h:
        return True
    else:
        return False

In [12]:
def reject_outliers(data, m=1):
    return data[abs(data - np.mean(data)) <= m * np.std(data)]

## Writing to Excel workbook

In [13]:
def addToExcel(dataname, data, row):
    col = 0

    worksheet.write(row, col, dataname)
    for content in data:
        col += 1
        worksheet.write(row, col, content)

## Matching the ratio for final data extraction 

Y-val data:
* The height of each bounding box is calculated by cluster count estimation method. 
* This ratio previously calculated is then used to calculate the <br> ``y_values :=  v_value = normalize_ratio x height of bounding box``.

In [14]:
def mergeRects(contours):
    rects = []
    rectsUsed = []

    # Just initialize bounding rects and set all bools to false
    for cnt in contours:
        rects.append(cv2.boundingRect(cnt))
        rectsUsed.append(False)

    # Sort bounding rects by x coordinate
    def getXFromRect(item):
        return item[0]

    rects.sort(key = getXFromRect)

    # Array of accepted rects
    acceptedRects = []

    # Merge threshold for x coordinate distance
    xThr = 5
    yThr = 5

    # Iterate all initial bounding rects
    for supIdx, supVal in enumerate(rects):
        if (rectsUsed[supIdx] == False):

            # Initialize current rect
            currxMin = supVal[0]
            currxMax = supVal[0] + supVal[2]
            curryMin = supVal[1]
            curryMax = supVal[1] + supVal[3]

            # This bounding rect is used
            rectsUsed[supIdx] = True

            # Iterate all initial bounding rects
            # starting from the next
            for subIdx, subVal in enumerate(rects[(supIdx+1):], start = (supIdx+1)):

                # Initialize merge candidate
                candxMin = subVal[0]
                candxMax = subVal[0] + subVal[2]
                candyMin = subVal[1]
                candyMax = subVal[1] + subVal[3]

                # Check if x distance between current rect
                # and merge candidate is small enough
                if (candxMin <= currxMax + xThr):

                    if not nearbyRectangle((candxMin, candyMin, candxMax - candxMin, candyMax - candyMin),
                                           (currxMin, curryMin, currxMax - currxMin, curryMax - curryMin), yThr):
                        break

                    # Reset coordinates of current rect
                    currxMax = candxMax
                    curryMin = min(curryMin, candyMin)
                    curryMax = max(curryMax, candyMax)

                    # Merge candidate (bounding rect) is used
                    rectsUsed[subIdx] = True
                else:
                    break

            # No more merge candidates possible, accept current rect
            acceptedRects.append([currxMin, curryMin, currxMax - currxMin, curryMax - curryMin])

    #for rect in acceptedRects:
    #    img = cv2.rectangle(img, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (121, 11, 189), 2)
    
    return acceptedRects

In [15]:
def mergeTextBoxes(textboxes):
    rects = []
    rectsUsed = []
    
    # Just initialize bounding rects and set all bools to false
    for box in textboxes:
        rects.append(box)
        rectsUsed.append(False)

    # Sort bounding rects by x coordinate
    def getXFromRect(item):
        return item[1][0]
    
    def getYFromRect(item):
        return item[1][1]

    rects.sort(key = lambda x: (getYFromRect, getXFromRect))

    #print(rects)
    #print("----------------------------------")
    
    # Array of accepted rects
    acceptedRects = []

    # Merge threshold for x coordinate distance
    xThr = 10
    yThr = 0

    # Iterate all initial bounding rects
    for supIdx, supVal in enumerate(rects):
        if (rectsUsed[supIdx] == False):

            # Initialize current rect
            currxMin = supVal[1][0]
            currxMax = supVal[1][0] + supVal[1][2]
            curryMin = supVal[1][1]
            curryMax = supVal[1][1] + supVal[1][3]
            currText = supVal[0]

            # This bounding rect is used
            rectsUsed[supIdx] = True

            # Iterate all initial bounding rects
            # starting from the next
            for subIdx, subVal in enumerate(rects[(supIdx+1):], start = (supIdx+1)):

                # Initialize merge candidate
                candxMin = subVal[1][0]
                candxMax = subVal[1][0] + subVal[1][2]
                candyMin = subVal[1][1]
                candyMax = subVal[1][1] + subVal[1][3]
                candText = subVal[0]

                # Check if x distance between current rect
                # and merge candidate is small enough
                if (candxMin <= currxMax + xThr):

                    if not nearbyRectangle((candxMin, candyMin, candxMax - candxMin, candyMax - candyMin),
                                           (currxMin, curryMin, currxMax - currxMin, curryMax - curryMin), yThr):
                        break

                    # Reset coordinates of current rect
                    currxMax = candxMax
                    curryMin = min(curryMin, candyMin)
                    curryMax = max(curryMax, candyMax)
                    currText = currText + ' ' + candText
                    
                    # Merge candidate (bounding rect) is used
                    rectsUsed[subIdx] = True
                else:
                    break

            # No more merge candidates possible, accept current rect
            acceptedRects.append([currText, (currxMin, curryMin, currxMax - currxMin, curryMax - curryMin)])

    #for rect in acceptedRects:
    #    img = cv2.rectangle(img, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (121, 11, 189), 2)
    
    return acceptedRects

In [16]:
def nearbyRectangle(current, candidate, threshold):
    (currx, curry, currw, currh) = current
    (candx, candy, candw, candh) = candidate
    
    currxmin = currx
    currymin = curry
    currxmax = currx + currw
    currymax = curry + currh
    
    candxmin = candx
    candymin = candy
    candxmax = candx + candw
    candymax = candy + candh
    
    # If candidate is on top, and is close
    if candymax <= currymin and candymax + threshold >= currymin:
        return True
    
    # If candidate is on bottom and is close
    if candymin >= currymax and currymax + threshold >= candymin:
        return True
    
    # If intersecting at the top, merge it
    if candymax >= currymin and candymin <= currymin:
        return True
    
    # If intersecting at the bottom, merge it
    if currymax >= candymin and currymin <= candymin:
        return True
    
    # If intersecting on the sides or is inside, merge it
    if (candymin >= currymin and
        candymin <= currymax and
        candymax >= currymin and
        candymax <= currymax):
        return True
    
    return False

In [17]:
def euclidean(v1, v2):
    return sum((p - q) ** 2 for p, q in zip(v1, v2)) ** .5

def angle_between(p1, p2):
    
    deltaX = p1[0] - p2[0]
    deltaY = p1[1] - p2[1]

    return math.atan2(deltaY, deltaX) / math.pi * 180
    
def RectDist(rectA, rectB):
    (rectAx, rectAy, rectAw, rectAh) = rectA
    (rectBx, rectBy, rectBw, rectBh) = rectB
    
    angle = angle_between((rectAx + rectAw / 2, rectAy + rectAh / 2), (rectBx + rectBw / 2, rectBy + rectBh / 2))
    
    if (angle >= -5 and angle <= 5) or (angle <= -175 and angle >= -185) or (angle >= 175 and angle <= 185):
        if (rectBx > rectAx + rectAw):
            return euclidean([rectAx + rectAw / 2, rectAy + rectAh / 2], [rectBx + rectBw / 2, rectBy + rectBh / 2])
        else:
            return sys.maxsize
    else:
        return sys.maxsize

In [18]:
def getProbableYLabels(image, contours, xaxis, yaxis):
    y_labels = []
    
    height, width, channels = image.shape
    
    (x1, y1, x2, y2) = xaxis
    (x11, y11, x22, y22) = yaxis
    
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, rect in enumerate(contours):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(contours[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
                    
    return image, maxList

## Saving y-values in our data excel sheet 

In [19]:
import ast, cv2, re, sys
import math
import json


images = []
texts = []

def getYVal(img_dir):
    yValueDict = {}
    for index, path in enumerate(Path(img_dir).iterdir()):
        if path.name.endswith('.png') or path.name.endswith('.jpg'):
            filepath = img_dir + "/" + path.name
            img = cv2.imread(filepath)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            xaxis, yaxis = detectAxes(filepath)
            
            for (x1, y1, x2, y2) in [xaxis]:
                xaxis = (x1, y1, x2, y2)

            for (x1, y1, x2, y2) in [yaxis]:
                yaxis = (x1, y1, x2, y2)

            image_text = images_text[path.name]
            img, x_labels, _, _, _, legends, legendBoxes = getProbableLabels(img, image_text, xaxis, yaxis)
            try:
                list_text, normalize_ratio = getRatio(path, xaxis, yaxis)
                print("[getYVal] legends: {0}".format(legends))
                print("[{0}] path: {1}, ratio: {2}".format(index, path.name, normalize_ratio), end='\n\n')

                # Color detection
                h, w, _ = np.shape(img)

                image = img.reshape((h * w, 3))

                # Remove white and near-by pixels
                threshold = 5
                for r in range(255 - threshold, 256):
                    for g in range(255 - threshold, 256):
                        for b in range(255 - threshold, 256):
                            image = image[np.where((image != [r, g, b]).any(axis = 1))]

                values, counts = np.unique(image, axis = 0, return_counts = True)

                greater_counts = np.where(counts >= 100)
                values, counts = values[greater_counts], counts[greater_counts]

                sort_indices = np.argsort(-counts)
                values, counts = values[sort_indices], counts[sort_indices]

                groups = []
                groupcounts = []

                for idx, value in enumerate(values):
                    grouped = False

                    for groupid, group in enumerate(groups):
                        for member in group:
                            r, g, b = member
                            vr, vg, vb = value

                            if (abs(vr.astype(np.int16) - r.astype(np.int16)) <= 5 and
                                abs(vg.astype(np.int16) - g.astype(np.int16)) <= 5 and
                                abs(vb.astype(np.int16) - b.astype(np.int16)) <= 5):
                                group.append(value)
                                groupcounts[groupid] += counts[idx]
                                grouped = True
                                break

                        if grouped:
                            break

                    if not grouped:
                        groups.append([value])
                        groupcounts.append(counts[idx])

                groupcounts = np.array(groupcounts)
                sort_indices = np.argsort(-groupcounts)
                new_groups = [groups[i] for i in sort_indices]
                groups = new_groups
                
                for i in range(len(groups[:4])):

                    img = cv2.imread(img_dir + "/" + path.name)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                    mask = None
                    for value in groups[i]:
                        COLOR_MIN = np.array([value[0], value[1], value[2]], np.uint8)
                        COLOR_MAX = np.array([value[0], value[1], value[2]], np.uint8)

                        if mask is None:
                            mask = cv2.inRange(img, COLOR_MIN, COLOR_MAX)
                        else:
                            mask = mask | cv2.inRange(img, COLOR_MIN, COLOR_MAX)

                    image = cv2.bitwise_and(img, img, mask = mask)
                    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, (3, 3))

                    edged = cv2.Canny(image, 0, 250)
                    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    contours = [contour for contour in contours if cv2.contourArea(contour) > 0.]

                    # Remove noisy ones!
                    if len(contours) == 0 or len(contours) > 100:
                        continue

                    contours = [cv2.approxPolyDP(contour, 3, True) for contour in contours]

                    rects = mergeRects(contours)
                    textBoxes = []
                    legends = []
                    
                    for rectBox in rects:
                        min_distance = sys.maxsize
                        closestBox = None
                        legendtext = None

                        for text, textBox in legendBoxes:    
                            if RectDist(rectBox, textBox) < min_distance:
                                closestBox = textBox
                                min_distance = RectDist(rectBox, textBox)
                                legendtext = text

                        if closestBox is not None:
                            textBoxes.append(closestBox)

                        if legendtext is not None:
                            legends.append(legendtext)

                    list_len = []
                    
                    for rect in rects:
                        list_len.append(float(rect[3]))

                    # y-values will be a product of the normalize ratio and each length              
                    y_val = [round(l* normalize_ratio, 1) for l in list_len]

                    if path.name in yValueDict:
                        if legends:
                            yValueDict[path.name].append([legends[-1]])
                        else:
                            yValueDict[path.name].append([''])
                            
                        yValueDict[path.name].append(y_val)
                    else:
                        if legends:
                            yValueDict[path.name] = [legends[-1]]
                        else:
                            yValueDict[path.name] = ['']
                            
                        yValueDict[path.name].append(y_val)
                        
                    for box in textBoxes:
                        (x, y, w, h) = box
                        
            except Exception as e:
                print(e)
                continue
            
    return yValueDict

In [20]:
yValueDict = getYVal(img_dir)

[get text-to-tick ratio] ticks_diff: [44.999994307756424, 44.000006437301636, 46.99999940395355, 44.000006437301636, 45.99999392032623, 44.000006437301636], text_diff: [15.0, 15.0, 15.0, 15.0, 15.0, 15.0]
[reject_outliers] ticks_diff: [44.99999431 44.00000644 44.00000644 45.99999392 44.00000644], text_diff: [15. 15. 15. 15. 15. 15.]
[getYVal] legends: ['Biocrude Oil', 'Co,', 'CH', 'H']
[0] path: 08813216-Figure1-1.png, ratio: 0.33632285858354183

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: ['A', 'B']
[1] path: 08813216-Figure4-1.png, ratio: nan



C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\users\dsand\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:62: RuntimeWar

[get text-to-tick ratio] ticks_diff: [34.000000562518835, 33.999999821186066, 33.99999389052391, 33.999999821186066, 35.00000435113907, 33.00000715255737, 32.99998342990875, 35.00001621246338, 33.999987959861755], text_diff: [0.1, 0.09999999999999998, 0.10000000000000003, 0.09999999999999998, 0.09999999999999998, 0.09999999999999998, 0.10000000000000009, 0.09999999999999998, 0.09999999999999998]
[reject_outliers] ticks_diff: [34.00000056 33.99999982 33.99999389 33.99999982 33.99998796], text_diff: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[getYVal] legends: ['SR', 'HSP', 'MSP', 'WSP']
[2] path: 1-s2.0-S0009250913008336-main-Figure10-1.png, ratio: 0.00294117678105061

[get text-to-tick ratio] ticks_diff: [34.00000225007534, 33.99999922513962, 31.999996423721313, 33.99999922513962, 32.0000085234642, 34.999994575977325, 19.000008463859558, 46.99999928474426, 31.999996423721313, 32.99999177455902], text_diff: [0.1, 0.1, 0.09999999999999998, 0.10000000000000003, 0.09999999999999998, 0.0999999999999

[get text-to-tick ratio] ticks_diff: [37.00000014156103, 36.00000411272049, 71.99999842047691, 36.99999278783798, 36.00000411272049], text_diff: [5.0, 5.0, 10.0, 5.0, 5.0]
[reject_outliers] ticks_diff: [37.00000014 36.00000411 36.99999279 36.00000411], text_diff: [5. 5. 5. 5.]
[getYVal] legends: ['Mcthanul', 'Fthanal', "I'ropanol", 'I-prepanul', 'ACelen']
[19] path: 1-s2.0-S0016236115009734-main-Figure4-1.png, ratio: 0.13698630028632014

[get text-to-tick ratio] ticks_diff: [40.99999885261059, 79.00000575184822, 79.9999868273735, 39.00001233816147], text_diff: [10.0, 20.0, 20.0, 10.0]
[reject_outliers] ticks_diff: [40.99999885 79.00000575], text_diff: [10. 20. 20. 10.]
[getYVal] legends: ['Methane', 'Ethane', 'Ethene', 'Propane', 'Propene', 'CO', 'CO2']
[20] path: 1-s2.0-S0016236115009734-main-Figure5-1.png, ratio: 0.24999999040737786

[get text-to-tick ratio] ticks_diff: [34.00000022351742, 33.99999734759331, 34.00000309944153, 34.00000309944153, 34.99999642372131, 34.00000309944153, 

[get text-to-tick ratio] ticks_diff: [23.000000584870577, 24.000000290572643, 23.999997839331627, 24.00000274181366, 22.999994456768036, 96.99999964237213, 23.000014066696167, 23.99998313188553, 23.000014066696167], text_diff: [10.0, 10.0, 10.0, 40.0, 10.0, 10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [23.00000058 24.00000029 23.99999784 24.00000274 22.99999446 23.00001407
 23.99998313 23.00001407], text_diff: [10. 10. 10. 10. 10. 10. 10. 10.]
[getYVal] legends: ['Bio-crude oil B Solid residue B Aqueous phase & Gas']
[37] path: 1-s2.0-S0016236117307172-main-Figure3-1.png, ratio: 0.425531898645011

[get text-to-tick ratio] ticks_diff: [26.0000004414469, 23.999998904764652, 24.000002160668373, 22.999996915459633, 25.00000089406967, 23.99999564886093, 25.00000089406967, 24.000008672475815, 23.000003427267075, 25.999980092048645, 24.0000216960907, 22.99999040365219, 25.00000089406967], text_diff: [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]
[reject_outliers] t

...

[get text-to-tick ratio] ticks_diff: [60.99999672174454, 181.99999737739563], text_diff: [60.0, 20.0]
[reject_outliers] ticks_diff: [ 60.99999672 181.99999738], text_diff: [60. 20.]
[getYVal] legends: ['MPa', 'MPa', 'MPa', 'MPa', 'MPa']
[1158] path: er.4843-Figure5-1.png, ratio: 0.3292181149904113

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: []
[1159] path: es304532c-Figure2-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: []
[1160] path: es304532c-Figure6-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: ['Others', 'O-Containing compounds', 'Amides', 'Amines', 'N-containing heterocycles', 'Quinolines', 'Pyridines']
[1161] path: Fan2018_Article_HydrothermalLiquefactionOfProt-Figure5-1.png, ratio: nan

[get text-to-tick ratio] tic

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: ['Wood', 'with KOH', 'Primary', 'char']
[1185] path: ie900964u-Figure6-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: []
[1186] path: ie900964u-Figure7-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [55.00000097230077, 54.99999974668026], text_diff: [0.4, 7.0]
[reject_outliers] ticks_diff: [55.00000097 54.99999975], text_diff: [0.4 7. ]
[getYVal] legends: []
[1187] path: ie9014809-Figure20-1.png, ratio: 0.06727272683301988

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: []
[1188] path: ie9014809-Figure21-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [64.00000209733844, 64.00000011920929, 67.9999988079071], text_diff: [0.6, 0.20000000000000007, 0.19999999999999996]
[reject_outliers]

[get text-to-tick ratio] ticks_diff: [51.00000185519457, 51.999995470047, 50.999996066093445, 52.00000870227814, 51.999995470047, 51.000009298324585], text_diff: [5.0, 5.0, 5.0, 5.0, 5.0, 5.0]
[reject_outliers] ticks_diff: [51.99999547 51.99999547 51.0000093 ], text_diff: [5. 5. 5. 5. 5. 5.]
[getYVal] legends: []
[1208] path: Onwudili2014_Chapter_HydrothermalGasificationOfBiom-Figure10.11-1.png, ratio: 0.09677419339953054

[get text-to-tick ratio] ticks_diff: [63.000000363215804, 62.00000247359276, 62.999994456768036, 62.00000247359276, 61.99998897314072], text_diff: [10.0, 10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [62.00000247 62.00000247 61.99998897], text_diff: [10. 10. 10. 10. 10.]
[getYVal] legends: ['No Catalyst', 'KCO3', 'Trona']
[1209] path: Onwudili2014_Chapter_HydrothermalGasificationOfBiom-Figure10.8-1.png, ratio: 0.16129032785264824

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: []
[1210

[get text-to-tick ratio] ticks_diff: [35.000001937150955, 35.99999758601189, 34.00000011920929, 34.99999576807022, 36.00000375509262, 34.00001245737076, 34.00000011920929, 34.99998342990875, 36.00001609325409, 34.00000011920929], text_diff: [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [35.00000194 34.99999577 34.99998343], text_diff: [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
[getYVal] legends: ['Glc%', 'HMF', 'OLeA+FA%', 'LE % A']
[1231] path: sc500686j-Figure5-1.png, ratio: 0.2857143370472747

[get text-to-tick ratio] ticks_diff: [68.99999931454659, 74.99999582767487, 209.00001049041748], text_diff: [8.0, 40.0, 10.0]
[reject_outliers] ticks_diff: [68.99999931 74.99999583], text_diff: [ 8. 10.]
[getYVal] legends: ['Oil yield', 'Solid residue']
[1232] path: Shin2019_Article_FateAndTransportOfEstrogenicCo-Figure3-1.png, ratio: 0.1250000042168218

[get text-to-tick ratio] ticks_diff: [60.00000174343586, 56.99999964237213, 60.999993324279785, 5

[get text-to-tick ratio] ticks_diff: [36.00000038743019, 36.99999779462814, 35.99999785423279, 36.0000079870224, 110.99998831748962], text_diff: [10.0, 0.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [36.00000039 36.99999779 35.99999785 36.00000799], text_diff: [10. 10. 10. 10.]
[getYVal] legends: ['costs']
[1251] path: wene.319-Figure6-1.png, ratio: 0.2758620613111754

[get text-to-tick ratio] ticks_diff: [53.00000309944153, 159.99999567866325], text_diff: [60.0, 20.0]
[reject_outliers] ticks_diff: [ 53.0000031  159.99999568], text_diff: [60. 20.]
[getYVal] legends: ['SS160', 'SB160', 'SS180', 'SB180', 'SS200', 'SB200']
[1252] path: Xie2019_Article_DewaterabilityEnhancementAndHe-Figure5-1.png, ratio: 0.37558685661468444

[get text-to-tick ratio] ticks_diff: [51.000000178813934, 50.999996066093445, 51.999995827674866, 52.000004053115845, 52.000004053115845, 51.000004291534424, 51.99998760223389, 51.000004291534424, 51.99998760223389], text_diff: [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0,

In [21]:
workbook = xlsxwriter.Workbook('FigureData1.xlsx')

for index, path in enumerate(Path(img_dir).iterdir()):
    if path.name.endswith('.png') or path.name.endswith('.jpg') or path.name.endswith('.jpeg'):
        if path.name in yValueDict:
            filepath = img_dir + "/" + path.name
            print("[{0}] path: {1}".format(index, path.name))
            
            image = cv2.imread(filepath)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            height, width, channels = image.shape
            xaxis, yaxis = detectAxes(filepath)
            y_text = []

            for (x1, y1, x2, y2) in [xaxis]:
                xaxis = (x1, y1, x2, y2)

            for (x1, y1, x2, y2) in [yaxis]:
                yaxis = (x1, y1, x2, y2)
                
            image_text = images_text[path.name]
            image, x_labels, y_labels, y_labels_list, y_text_list, legends, _ = getProbableLabels(image,
                                                                                                  image_text,
                                                                                                  xaxis,
                                                                                                  yaxis)
            
            # Sort bounding rects by y coordinate
            def getYFromRect(item):
                return item[1][1]

            y_labels_list.sort(key = getYFromRect)
            y_text_list.sort(key = getYFromRect, reverse=True)
            
            for text, (textx, texty, w, h) in y_text_list:
                y_text.append(text)

            # Append doi values for each image.
            if path.name[:-16]+'.pdf' in doidata['Dois']:
                doi = doidata['Dois'][path.name[:-16]+'.pdf']
            else:
                doi = ''
             
            # Write to Excel
            worksheet = workbook.add_worksheet()            
            
            addToExcel("doi",doi, 0)
            addToExcel("file name", [path.name[::-4] + ".pdf"], 1)
            addToExcel("x-labels", x_labels, 2)
            addToExcel("y-text", y_text, 3)
            addToExcel("y-labels", y_labels, 4)
            addToExcel("legends", legends, 5)
            
            for col_num, data in enumerate(yValueDict[path.name]):
                print(col_num, data)
                if type(data) != list:
                    worksheet.write(9 + col_num, 0, "Y-values:" + "\n\n" + str(data))
                else:
                    for pos,d in enumerate(data):
                        worksheet.write(9 + col_num, pos, str(d))

            # Print the output here!
            print("file name    :  ", path.name)
            print("doi          :  ", doi)
            print("x-labels     :  ", x_labels)
            print("y-text       :  ", y_text)
            print("y-labels     :  ", y_labels)
            print("legends      :  ", legends)
            print("Y- Values    :  ", ' '.join([str(elem) for elem in yValueDict[path.name]]), end = "\n\n")

            # Insert the image
            worksheet.insert_image('J21', filepath)

# Close the excel workbook!
workbook.close()

[0] path: 08813216-Figure1-1.png
0 CH
1 [4.4, 35.3, 0.7, 35.0, 28.9, 30.9, 37.7]
2 ['Biocrude Oil']
3 [4.7, 26.9, 25.2, 26.6, 22.2, 30.3, 27.2]
4 ['']
5 [17.2, 20.9, 25.9, 4.7, 1.7, 20.2, 19.5]
6 ['']
7 [21.2, 18.8, 17.8, 18.5, 15.5, 4.7, 15.5]
file name    :   08813216-Figure1-1.png
doi          :   
x-labels     :   ['270 0', "290'C", '310 C', "330'C", '350']
y-text       :   ['perce', 'of']
y-labels     :   ['105', '90', '75', '60', '45', '30', '15']
legends      :   ['Biocrude Oil', 'Co,', 'CH', 'H']
Y- Values    :   CH [4.4, 35.3, 0.7, 35.0, 28.9, 30.9, 37.7] ['Biocrude Oil'] [4.7, 26.9, 25.2, 26.6, 22.2, 30.3, 27.2] [''] [17.2, 20.9, 25.9, 4.7, 1.7, 20.2, 19.5] [''] [21.2, 18.8, 17.8, 18.5, 15.5, 4.7, 15.5]

[1] path: 08813216-Figure4-1.png
0 
1 [nan]
2 ['']
3 [nan, nan]
4 ['B']
5 [nan]
file name    :   08813216-Figure4-1.png
doi          :   
x-labels     :   ['5', '10', '15', '20', '25', '30', '35', '40', '45']
y-text       :   ['Others', 'N-containing', '&', 'Alcohols', 'Pheno

0 
1 [0.0, 0.1, 0.1, 0.1, 0.2, 0.1, 15.1, 0.1, 4.0, 0.1, 0.4, 1.3, 0.5, 0.1, 0.2, 0.1, 0.2, 0.1, 0.4, 0.1, 0.1, 0.1, 0.1, 0.4, 0.4, 4.4, 0.1, 0.5, 1.5, 0.5, 0.1, 0.2, 0.1, 0.3, 0.1, 0.1, 0.1, 0.1, 0.1, 4.1, 0.1, 1.6, 0.1, 0.1, 0.1, 3.9, 0.1, 0.1, 1.7, 0.1]
2 ['']
3 [0.1, 0.3, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.1, 0.6, 0.1, 0.1, 0.1, 0.2, 0.3, 0.2, 0.5, 0.5, 0.4, 0.5, 0.2, 0.5, 0.5, 0.1, 0.5, 0.0, 0.2, 0.2, 0.1, 0.2, 0.4, 0.3, 0.2, 0.4, 0.2, 0.8]
file name    :   1-s2.0-S0016236114000477-main-Figure8-1.png
doi          :   
x-labels     :   ['K2CO3', 'K2CO3', 'K2C03', 'K2C03']
y-text       :   ['Fatty', 'acid', 'content', 's']
y-labels     :   ['14', '12', '10', '8', '4', '2']
legends      :   []
Y- Values    :    [0.0, 0.1, 0.1, 0.1, 0.2, 0.1, 15.1, 0.1, 4.0, 0.1, 0.4, 1.3, 0.5, 0.1, 0.2, 0.1, 0.2, 0.1, 0.4, 0.1, 0.1, 0.1, 0.1, 0.4, 0.4, 4.4, 0.1, 0.5, 1.5, 0.5, 0.1, 0.2, 0.1, 0.3, 0.1, 0.1, 0.1, 0.1, 0.1, 4.1, 0.1, 1.6, 0.1, 0.1, 0.1, 3.9, 0.1, 0.1

0 Residual Solids
1 [4.7, 30.7, 33.3, 8.6, 5.3, 2.3, 48.4]
2 ['AC-Purified Lipids']
3 [2.1, 3.5, 83.5, 11.9, 2.3, 2.6]
file name    :   1-s2.0-S0016236116302447-main-Figure2-1.png
doi          :   
x-labels     :   []
y-text       :   ['ConCEn', 'W']
y-labels     :   ['90', '80', '70', '60', '50', '40', '30', '20', '10', '0']
legends      :   ['Raw Algae', 'Residual Solids', 'Crude Lipids', 'AC-Purified Lipids']
Y- Values    :   Residual Solids [4.7, 30.7, 33.3, 8.6, 5.3, 2.3, 48.4] ['AC-Purified Lipids'] [2.1, 3.5, 83.5, 11.9, 2.3, 2.6]

[28] path: 1-s2.0-S0016236116302447-main-Figure3-1.png
0 Raw Algae
1 [0.3, 0.0, 1.3]
2 ['']
3 [0.0, 0.1, 0.9]
4 ['Crude Lipids']
5 [0.0, 0.2, 0.7]
file name    :   1-s2.0-S0016236116302447-main-Figure3-1.png
doi          :   
x-labels     :   ['Magnesium', 'Phosphorus']
y-text       :   ['Concen']
y-labels     :   ['1.4', '1.2', '1.0', '0.8', '0.6', '0.4', '0.2+ -', '0.0']
legends      :   ['Raw Algae', 'Crude Lipids', 'T']
Y- Values    :   Raw Algae 

0 -treatment
1 [1.8, 41.7, 2.7, 5.3, 26.6]
2 ['']
3 [2.1, 0.5, 0.5, 0.5, 0.5, 60.2, 2.1, 1.8, 0.9, 1.1, 1.4, 1.8, 1.4, 1.4]
4 ['-treatment']
5 [1.4, 0.9, 1.8]
file name    :   1-s2.0-S0016236117307172-main-Figure2-1.png
doi          :   
x-labels     :   ['Mass', 'loss', 'Ash', 'Carbohydrates', 'Protein']
y-text       :   ['Yie']
y-labels     :   ['60', '50', '40', '30', '20', '10']
legends      :   ['E Dilute acid', '-treatment']
Y- Values    :   -treatment [1.8, 41.7, 2.7, 5.3, 26.6] [''] [2.1, 0.5, 0.5, 0.5, 0.5, 60.2, 2.1, 1.8, 0.9, 1.1, 1.4, 1.8, 1.4, 1.4] ['-treatment'] [1.4, 0.9, 1.8]

[37] path: 1-s2.0-S0016236117307172-main-Figure3-1.png
0 
1 [1.3, 1.3, 0.9, 2.6, 0.9, 1.7, 2.1, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 123.0, 4.3, 3.0, 2.1, 2.1, 4.7, 4.3, 4.3, 2.1]
file name    :   1-s2.0-S0016236117307172-main-Figure3-1.png
doi          :   
x-labels     :   ['Raw', 'microalgae', 'PA', '(NaOH/urea)', 'PA', '(dilute', 'acid)']
y-text       :   ['yie']
y-labels     :   

0 
1 [35.4, 39.8, 0.5, 42.5, 46.9, 43.4]
2 ['']
3 [33.9, 25.2, 0.8, 0.1, 18.3, 10.6, 17.3]
file name    :   1-s2.0-S0016236118304496-main-Figure5-1.png
doi          :   
x-labels     :   ['No', 'catalyst', 'HZSM-5', 'Co/HZSM-5', 'Co-Zn/HZSM-5', 'Zn/HZSM-5']
y-text       :   ['C', '(vo']
y-labels     :   ['50', '45', '40', '35', '30', '25', '20', '15', '10', '5', '0']
legends      :   []
Y- Values    :    [35.4, 39.8, 0.5, 42.5, 46.9, 43.4] [''] [33.9, 25.2, 0.8, 0.1, 18.3, 10.6, 17.3]

[50] path: 1-s2.0-S0016236118304502-main-Figure4-1.png
0 
1 [48.3, 61.5, 63.2, 6.0, 59.8, 56.0, 53.6, 1.3, 53.8, 37.0, 61.3, 67.2, 52.8, 64.9, 43.2, 0.9, 37.4, 73.2]
2 ['Char']
3 [42.3, 0.9, 0.2, 34.0, 34.9, 32.8, 41.9, 38.1, 39.6, 48.1, 33.8, 28.3, 39.4, 20.0, 38.9, 40.0, 20.2]
file name    :   1-s2.0-S0016236118304502-main-Figure4-1.png
doi          :   
x-labels     :   []
y-text       :   ['(current', 'Chlorella', 'study)', 'v.', 'raw', 'W']
y-labels     :   ['100.0', '80.0', '60.0', '40.0', '20.0', 

...

0 
1 [nan, nan, nan, nan, nan]
2 ['']
3 [nan, nan, nan, nan, nan]
4 ['']
5 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
file name    :   ie5020684-Figure5-1.png
doi          :   
x-labels     :   ['He', '1', 'bar', 'H2', '1', 'bar', 'H2', '10', 'bar', 'H2', '30', 'bar']
y-text       :   []
y-labels     :   []
legends      :   []
Y- Values    :    [nan, nan, nan, nan, nan] [''] [nan, nan, nan, nan, nan] [''] [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

0 
1 [0.1, 0.1, 8.5, 34.8, 31.0, 26.4, 0.1, 24.5, 22.5, 22.4, 20.5, 0.1, 0.8, 22.7, 0.7]
file name    :   Minarick2011_Article_ProductAndEconomicAnalysisOfDi-Figure1-1.png
doi          :   
x-labels     :   ['Fresh', 'Solids', 'after', 'Solids', 'after', 'Coarse', 'filter', 'Shallow', 'pit', 'Shallow', 'pit', 'Shallow', 'pit']
y-text       :   ['Refined', 'O', 'e']
y-labels     :   ['45.0', '40.0', '35.0', '30.0', '25.0', '20.0', '15.0', '10.0', '5.0', '0.0']
legends      :   []
Y- Values    :    [0.1, 0.1, 8.5, 34.8, 31.0, 26.4, 0.1, 24.5, 22.5, 22.4, 20.5, 0.1, 0.8, 22.7, 0.7]

[1202] path: Minarick2011_Article_ProductAndEconomicAnalysisOfDi-Figure2-1.png
0 manure from behind
1 [0.3, 0.9, 0.1, 0.3, 0.3, 0.4, 0.3, 0.1, 0.4, 0.4, 0.1, 0.3, 0.3, 0.7, 1.0, 0.3, 0.9, 0.6, 0.3, 0.1, 0.3, 0.3, 0.3, 2.8, 10.8, 0.4, 0.3, 0.3, 0.3, 0.3, 2.5, 10.7, 8.2, 8.5, 9.5, 8.3, 8.6]
file name    :   Minarick2011_Article_ProductAndEconomicAnalysisOfDi-Figure2-1.png
doi          :   
x-labels     :   []
y-

0 LP
1 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
file name    :   Phenolic Antioxidant Production by Hydrothermal Liquefaction of Lignin-Figure5-1.png
doi          :   
x-labels     :   ['150', 'C', '130', 'C']
y-text       :   ['Thermal', 'N', 'oxidation', 'stability', 'time', '(min)']
y-labels     :   ['9']
legends      :   ['Tung oil with', 'LP']
Y- Values    :   LP [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]

[1214] path: Phusunti2017_Article_SubcriticalWaterHydrolysisOfMi-Figure2-1.png
0 
1 [1.6, 0.3, 1.6, 1.9]
file name   

0 K,CO,
1 [0.7, 0.6, 0.7, 5.2, 4.9, 3.1, 12.6, 0.9, 2.8, 13.0, 5.4, 3.0, 0.9, 2.1, 1.5, 0.9, 1.5, 0.1, 1.2, 1.2, 40.2, 0.9, 1.0, 2.4, 1.6, 3.1, 3.1, 5.2, 0.7, 2.2, 0.7, 1.2, 1.2, 0.6, 0.7, 1.2]
file name    :   Singh2015_Article_HydrothermalLiquefactionOfAgri-Figure8-1.png
doi          :   
x-labels     :   ['5-28', 'ppm', '28-55', 'ppm', '55-95', 'ppm', '95-165', 'ppm', '165-180', 'ppm', '180-215', 'ppm']
y-text       :   ['D', 'Carbon', '%']
y-labels     :   ['50', '1i', '10']
legends      :   ['THERMAL', 'KOH', 'K,CO,']
Y- Values    :   K,CO, [0.7, 0.6, 0.7, 5.2, 4.9, 3.1, 12.6, 0.9, 2.8, 13.0, 5.4, 3.0, 0.9, 2.1, 1.5, 0.9, 1.5, 0.1, 1.2, 1.2, 40.2, 0.9, 1.0, 2.4, 1.6, 3.1, 3.1, 5.2, 0.7, 2.2, 0.7, 1.2, 1.2, 0.6, 0.7, 1.2]

[1237] path: slct.201601903-Figure2-1.png
0 nitro-compounds
1 [1.8, 0.5, 0.2, 18.3, 2.8, 0.2, 0.3, 0.5, 0.2, 1.5, 0.5, 0.5, 0.7, 0.2, 0.3, 0.5, 0.8, 0.8, 0.2, 0.7, 2.1, 2.0, 1.2, 17.2, 0.3, 0.2, 13.9, 0.2, 1.0, 1.7, 1.5, 0.8, 1.7, 7.3, 1.2, 1.0, 0.7, 1.2, 1.2, 8.

0 Light Oil
1 [5.3, 1.0, 17.6, 1.0]
2 ['']
3 [7.7, 5.0, 10.6, 1.0, 1.2]
file name    :   Wagner2018_Article_HydrothermalConversionOfLipid--Figure2-1.png
doi          :   
x-labels     :   ['O', 'vol%', 'IPA', '50', 'vol%', 'IPA']
y-text       :   ['BIOCI', 'yie']
y-labels     :   ['35%', '30%', '25%', '20%', '15%', '10%', '5%', '0%']
legends      :   ['Light Oil', 'Heavy Oil']
Y- Values    :   Light Oil [5.3, 1.0, 17.6, 1.0] [''] [7.7, 5.0, 10.6, 1.0, 1.2]

[1250] path: wene.319-Figure5-1.png
0 
1 [47.8, 6.7, 50.0, 52.2]
2 ['']
3 [37.8, 40.6, 6.7, 41.7]
4 ['']
5 [6.7, 14.4, 14.4, 14.4]
file name    :   wene.319-Figure5-1.png
doi          :   
x-labels     :   ['LLE', 'HDO', 'reators', 'Chemical', 'storage']
y-text       :   ['O', 'SM', 'Instanled']
y-labels     :   ['140', '120', '100', '80', '60', '40', '20']
legends      :   ['capital costs']
Y- Values    :    [47.8, 6.7, 50.0, 52.2] [''] [37.8, 40.6, 6.7, 41.7] [''] [6.7, 14.4, 14.4, 14.4]

[1251] path: wene.319-Figure6-1.png
0 cost